In [ ]:
DATA = '/media/data/ioc/data'

%cd ..
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
from rts.settings import DATABASE_URL
from rts.db.dao import DataAccessObject
from rts.pipelines.ioc import PipelineIOC
from rts.utils import dataframe_from_hdf5

DataAccessObject().connect(DATABASE_URL)


In [ ]:
# metadata = '../data/metadata.hdf5'
df = dataframe_from_hdf5(DATA, 'metadata')

In [ ]:
# This preprocessing step should be outside of the pipeline code and come with the dataframe
def find_mp4_files(root_folder):
    import os
    mp4_files = []

    for foldername, _, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith('.mp4'):
                mp4_files.append(os.path.join(foldername, filename))

    return mp4_files

mp4_files = find_mp4_files("/media/data/ioc/videos/")
file_map = {}
for mp4 in mp4_files:
    file_map[mp4.split('/')[-1].split('.')[0]] = mp4

df['path'] = df['guid'].apply(lambda x: file_map[x] if x in file_map else None)
# we need a path for each video, otherwise we can't process
df = df[df['path'].notna()]

In [ ]:
# Process a single video and upload clips. This funcction would also take a dataframe that has more than one video
pipeline = PipelineIOC(run_notebook=True)

pipeline.ingest(df[df.guid == 'MDA42810489'])

In [ ]:
def find_mp4_files(root_folder):
    import os
    mp4_files = []

    for foldername, _, filenames in os.walk(root_folder):
        for filename in filenames:
            # if filename.endswith('.mp4'):
            mp4_files.append(os.path.join(foldername, filename))

    return mp4_files

metadata_files = find_mp4_files("/media/data/ioc/metadata/")
file_map = {}
for mp4 in mp4_files:
    file_map[mp4.split('/')[-1].split('.')[0]] = mp4

In [ ]:
for mp4 in mp4_files:
    if 'MDA42710309' in mp4:
        print(mp4)  

In [ ]:
for mp4 in mp4_files:
    if 'MDA42710309' in mp4:
        print(mp4)  

In [ ]:
len(metadata_files), len(mp4_files)

In [ ]:
# clean up metadata and mp4 file IDs
metadata_files = [x.split('/')[-1] for x in metadata_files]
metadata_files.sort()
mp4_files = [x.split('/')[-1] for x in mp4_files]
mp4_files.sort()
mp4_files = [x.split('.')[0] for x in mp4_files]
metadata_files = [x.split('.')[0] for x in metadata_files]
metadata_files = [x.split('_')[-1] for x in metadata_files]
# count overlap between metadata and mp4 files
len(set(metadata_files).intersection(set(mp4_files)))